# Toy example to demonstrate the use of mescal

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pickle
from mescal import *
import bw2data as bd
import bw2io as bi
from utils import *
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
ei_version = '3.10.1'

In [ ]:
# bi.backup.restore_project_directory(
#     fp='./export/backup/brightway2-project-ecoinvent3.10.1-paper_mescal-backup.15-August-2025-11-18AM.tar.gz',
#     project_name=f'ecoinvent{ei_version}-paper_mescal',
#     overwrite_existing=True,
# )

In [ ]:
# AMPL licence
path_to_ampl_licence = r'C:\Users\matth\ampl' # Path to the AMPL license file
os.environ['PATH'] = path_to_ampl_licence+':'+os.environ['PATH']

In [ ]:
path_model = './data/esm/' # Path to the energy system model
path_model_lca = './data/esm/lca/'
path_inputs = './data/lca/' # Path to the LCA data
path_results = './results/' # Path to the results

## Generating LCA impact scores

In [ ]:
mapping = pd.read_csv(path_inputs+'mapping.csv')
unit_conversion = pd.read_excel(path_inputs+'unit_conversion.xlsx')
techno_compositions = pd.read_csv(path_inputs+'technology_compositions.csv')
efficiency = pd.read_csv(path_inputs+'efficiency.csv')
lifetime = pd.read_csv(path_inputs+'lifetime.csv')
mapping_es_flows_to_cpc = pd.read_csv(path_inputs+'mapping_esm_flows_to_CPC.csv')
impact_abbrev = pd.read_csv(path_inputs+'impact_abbrev.csv')
model = pd.read_csv(path_inputs+'model.csv')

In [ ]:
df_si_paper = pd.merge(mapping, unit_conversion, on=['Name', 'Type']).drop(columns=['Unit', 'Database']).rename(columns={'LCA': 'LCA unit', 'ESM': 'ESM unit', 'Name': 'Technology', 'Product': 'LCI dataset product', 'Activity': 'LCI dataset activity', 'Location': 'LCI dataset location', 'Value': 'Conversion factor', 'Type': 'Life cycle phase'})
df_si_paper['Technology'] = df_si_paper['Technology'].apply(lambda x: tech_name_dict[x] if x in tech_name_dict.keys() else x)
df_si_paper['Technology'] = df_si_paper['Technology'].replace('WIND_ONSHORE_CONNECTION', 'Onshore wind (connection part)')
df_si_paper['Technology'] = df_si_paper['Technology'].replace('WIND_ONSHORE_TURBINE', 'Onshore wind (turbine part)')
df_si_paper.drop(columns=['Year'], inplace=True)
# df_si_paper.to_csv('./data/lca/SI_mapping.csv', index=False)

In [ ]:
# Set up your Brightway project
bd.projects.set_current(f'ecoinvent{ei_version}-paper_mescal')

In [ ]:
name_main_database_2050 = f'ecoinvent_cutoff_{ei_version}_image_SSP2-Base_2050'
esm_db_name = 'Tatooine'

In [ ]:
main_db_2050 = Database(name_main_database_2050, create_pickle=True)

In [ ]:
# Add CPC categories to the main database
main_db_2050.add_CPC_categories()

In [ ]:
ranking_best_ecoinvent_locations = ['CHN', 'CN', 'RoW', 'World', 'GLO']

In [ ]:
esm = PathwayESM(
    # Mandatory inputs
    time_steps=[
        {'year': 2050, 'model': model, 'lifetime':lifetime, 'main_database': main_db_2050},
    ],
    mapping=mapping.copy(deep=True),
    unit_conversion=unit_conversion,
    mapping_esm_flows_to_CPC_cat=mapping_es_flows_to_cpc,
    esm_db_name=esm_db_name,
    esm_location='CHN',

    # Optional inputs
    technology_compositions=techno_compositions,
    efficiency=efficiency,
    regionalize_foregrounds='all',
    locations_ranking=ranking_best_ecoinvent_locations,
    accepted_locations=['CHN', 'CN'],
    results_path_file=path_results,
    biosphere_db_name='ecoinvent-3.10.1-biosphere',
    remove_double_counting_to=['Operation'],
)

In [ ]:
esm.check_inputs()

In [ ]:
esm.change_location_mapping_file()
esm.mapping.to_csv('data/lca/mapping.csv', index=False) # Save the mapping file with updated locations

In [ ]:
missing_flows = main_db_2050.test_mapping_file(esm.mapping[esm.mapping.Year == 2050])

In [ ]:
esm.create_esm_database()

In [ ]:
methods = ['IMPACT World+ Midpoint 2.1 for ecoinvent v3.10', 'IMPACT World+ Damage 2.1 for ecoinvent v3.10']

In [ ]:
# Contribution analysis of elementary flows for total human health and total ecosystem quality
R_long, contrib_analysis, df_req_technosphere = esm.compute_impact_scores(
    methods=methods,
    impact_abbrev=impact_abbrev,
    contribution_analysis='emissions',
    contribution_analysis_limit_type='number',
    contribution_analysis_limit=30,
    req_technosphere=True,
)
R_long.to_csv(f'{path_results}impact_scores.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]
contrib_analysis.to_csv(f'{path_results}contribution_analysis.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]
df_req_technosphere.to_csv(f'{path_results}required_technosphere_flows.csv', index=False)

In [ ]:
# Direct emissions
R_long_direct, _, _ = esm.compute_impact_scores(
    methods=methods,
    impact_abbrev=impact_abbrev,
    assessment_type='direct emissions',
)
R_long_direct.to_csv(f'{path_results}impact_scores_direct.csv', index=False) # [impact / kW(h) or pkm(/h) or tkm(/h)]

In [ ]:
metadata = {
    'ecoinvent_version': ei_version,
    'year': '2050',
    'iam': 'image',
    'ssp_rcp': 'SSP2-Base',
}

In [ ]:
specific_lcia_abbrev = ['m_CCS', 'TTHH', 'TTEQ']

In [ ]:
# Create .dat file
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=methods,
    specific_lcia_abbrev=specific_lcia_abbrev,
    impact_abbrev=impact_abbrev,
    path=path_model_lca,
    metadata=metadata,
    file_name='techs_lca',
)

In [ ]:
# Create the .mod file
esm.generate_mod_file_ampl(
    lcia_methods=methods,
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=specific_lcia_abbrev,
    path=path_model_lca,
    metadata=metadata,
    file_name='objectives_lca',
)

## First iteration: single objective optimization with a snapshot ESM

In [ ]:
impact_scores = pd.read_csv(path_results+'impact_scores.csv')
impact_scores.Impact_category = impact_scores.Impact_category.apply(lambda x: ast.literal_eval(x))

In [ ]:
list_esm_results_f_mult = []
list_esm_results_annual_res = []
list_esm_results_annual_prod = []
list_main_variables_results = []

for obj in ['TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']:

    es = run_esm(obj+'[2050]', returns='model')
    results = es.calc()

    df_f_mult, df_annual_prod, df_annual_res = get_impact_scores(
        df_results=results,
        df_impact_scores=impact_scores,
        impact_category=list(impact_scores['Impact_category'].unique()),
    )

    df_f_mult['Run'] = obj
    df_annual_prod['Run'] = obj
    df_annual_res['Run'] = obj

    # we only keep the results for 2050
    df_f_mult = df_f_mult[df_f_mult['index1'] == 2050]
    df_annual_prod = df_annual_prod[df_annual_prod['index1'] == 2050]
    df_annual_res = df_annual_res[df_annual_res['index1'] == 2050]

    df_f_mult.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)
    df_annual_prod.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)
    df_annual_res.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)

    # we keep the first value (i.e., the one for 2050)
    total_cost = results.variables['TotalCost'].TotalCost.values[0]
    total_m_ccs = results.variables['TotalLCIA_m_CCS'].TotalLCIA_m_CCS.values[0]
    total_tthh = results.variables['TotalLCIA_TTHH'].TotalLCIA_TTHH.values[0]
    total_tteq = results.variables['TotalLCIA_TTEQ'].TotalLCIA_TTEQ.values[0]

    list_main_variables_results.append([obj, total_cost, total_m_ccs, total_tthh, total_tteq])

    list_esm_results_f_mult.append(df_f_mult)
    list_esm_results_annual_prod.append(df_annual_prod)
    list_esm_results_annual_res.append(df_annual_res)

esm_results_f_mult = pd.concat(list_esm_results_f_mult)
esm_results_annual_prod = pd.concat(list_esm_results_annual_prod)
esm_results_annual_res = pd.concat(list_esm_results_annual_res)
main_variables_results = pd.DataFrame(data=list_main_variables_results, columns=['Objective', 'TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ'])

In [ ]:
# lyrio = results.parameters['layers_in_out'].reset_index()
# lyrio = lyrio[lyrio.layers_in_out != 0].drop(columns=['Run', 'index2']).rename({'index0': 'Name', 'index1': 'Flow', 'layers_in_out': 'Amount'}, axis=1)
# lyrio.to_csv(path_inputs+'model.csv', index=False)

### Energy input for local production of new infrastructures

In [ ]:
main_db = main_db_2050 + esm.esm_db
main_db_as_dict_code = main_db.db_as_dict_code

In [ ]:
def activity_main_info(database, code):
    act = main_db_as_dict_code[(database, code)]
    name = act['name']
    reference_product = act['reference product']
    location = act['location']
    return name, reference_product, location

In [ ]:
df_req_technosphere[['Technosphere flow name', 'Technosphere flow product', 'Technosphere flow location']] = df_req_technosphere.apply(lambda row: activity_main_info(row['Technosphere flow database'], row['Technosphere flow code']), axis=1, result_type='expand')

In [ ]:
df_req_technosphere_infra_esm_loc = df_req_technosphere[
    (df_req_technosphere['Type'] == 'Construction')
    & (df_req_technosphere['Technosphere flow location'] == esm.esm_location)
    & (df_req_technosphere['Technosphere flow name'].str.contains('market group for electricity, high voltage'))
]

In [ ]:
df_req_technosphere_infra_esm_loc = df_req_technosphere_infra_esm_loc.merge(esm_results_f_mult[['Name', 'F_Mult', 'Run']], how='left', on='Name')

In [ ]:
# GWh = [GWh / GW] * [GW]
df_req_technosphere_infra_esm_loc['Local electricity for infrastructure [GWh]'] = df_req_technosphere_infra_esm_loc['Amount'] * df_req_technosphere_infra_esm_loc['F_Mult']

In [ ]:
df_req_technosphere_infra_esm_loc.groupby('Run').sum()[['Local electricity for infrastructure [GWh]']]

## Create new LCI datasets from the ESM results

In [ ]:
esm_results = esm_results_annual_prod.rename(columns={'Annual_Prod': 'Production'})[['Name', 'Production', 'Run', 'Year']]
esm_results = esm_results.merge(esm_results_f_mult[['Name', 'F_Mult', 'Run', 'Year']], on=['Name', 'Run', 'Year'], how='left').rename(columns={'F_Mult': 'Capacity'})

### With harmonization

In [ ]:
for run in ['TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']:
    esm.create_new_database_with_esm_results(
        esm_results=esm_results[esm_results['Run'] == run],
        remove_background_construction_flows=False,
        harmonize_efficiency_with_esm=True,
        harmonize_capacity_factor_with_esm=True,
        esm_results_db_name=f'{esm.esm_db_name}_results_{run.replace("Total", "").replace("LCIA_", "").lower()}',
        name_capacity_factor_difference_file=f'capacity_factor_differences_{run.replace("Total", "").replace("LCIA_", "").lower()}'
    )

### Without harmonization

In [ ]:
for run in ['TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']:
    esm.create_new_database_with_esm_results(
        esm_results=esm_results[esm_results['Run'] == run],
        remove_background_construction_flows=False,
        harmonize_efficiency_with_esm=False,
        harmonize_capacity_factor_with_esm=False,
        esm_results_db_name=f'{esm.esm_db_name}_results_{run.replace("Total", "").replace("LCIA_", "").lower()}_wo_h',
        name_capacity_factor_difference_file=f'capacity_factor_differences_{run.replace("Total", "").replace("LCIA_", "").lower()}_wo_h'
    )

### Generate new metrics from the SOO results

In [ ]:
for run in ['TotalCost', 'TotalLCIA_TTEQ', 'TotalLCIA_TTHH', 'TotalLCIA_m_CCS']:

    for harmonized in [True, False]:

        if harmonized:
            suffix = f'{run.replace("Total", "").replace("LCIA_", "").lower()}'
        else:
            suffix = f'{run.replace("Total", "").replace("LCIA_", "").lower()}_wo_h'

        esm_results_db_name = f'{esm_db_name}_results_{suffix}_2050'
        new_esm_db_name = f'{esm_db_name}_2050_{suffix}'
        impact_scores_df_name = f'impact_scores_{suffix}'
        dat_file_name = f'techs_lca_{suffix}'

        main_db_2050 = Database(name_main_database_2050, create_pickle=True)
        main_db_2050.add_CPC_categories()

        esm = PathwayESM(
            # Mandatory inputs
            time_steps=2*[{'year': 2050, 'model': model, 'main_database': main_db_2050}],
            mapping=mapping.copy(deep=True),
            unit_conversion=unit_conversion,
            mapping_esm_flows_to_CPC_cat=mapping_es_flows_to_cpc,
            esm_db_name=esm_db_name,
            esm_location='CHN',

            # Optional inputs
            technology_compositions=techno_compositions,
            lifetime=lifetime,
            efficiency=efficiency,
            regionalize_foregrounds='all',
            locations_ranking=ranking_best_ecoinvent_locations,
            accepted_locations=['CHN', 'CN'],
            results_path_file=path_results,
            biosphere_db_name='ecoinvent-3.10.1-biosphere',
        )

        # Connecting the results of the ESM to the ESM database
        esm.connect_esm_results_to_database(
            esm_results_db_name=esm_results_db_name,
            specific_db_name=f'{esm.esm_db_name}_2050',
            create_new_db=True,
            new_db_name=new_esm_db_name,
            locations=['CHN', 'CN', 'World', 'GLO', 'RoW'],
            update_exchanges_based_on_activity_name=False,
        )

        # Connecting the results of the ESM to the main database (i.e., premise database)
        esm.connect_esm_results_to_database(
            esm_results_db_name=esm_results_db_name,
            create_new_db=True,
            new_db_name=f'{esm.main_database_name}_connected_{suffix}',
            locations=['CHN', 'CN', 'World', 'GLO', 'RoW'],
            update_exchanges_based_on_activity_name=False,
        )

        # relinking the newly created ESM database to the updated main database
        Database(new_esm_db_name).relink(
            name_database_unlink=esm.main_database_name,
            name_database_relink=f'{esm.main_database_name}_connected_{suffix}',
            write=True,
        )

        esm.time_steps = [{'year': 2050, 'model': model, 'main_database': main_db_2050}]  # only compute lca metrics once

        # Computing the LCA indicators with the new ESM database
        R_long, contribution_analysis, _ = esm.compute_impact_scores(
            esm_db_name=new_esm_db_name,
            methods=methods,
            impact_abbrev=impact_abbrev,
            specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
        )

        R_long.to_csv(f'{path_results}2050/{impact_scores_df_name}.csv', index=False)

        # R_long = pd.read_csv(f'{path_results}2050/{impact_scores_df_name}.csv')

        # Create .dat file
        esm.normalize_lca_metrics(
            R=R_long,
            mip_gap=1e-6,
            lcia_methods=methods,
            specific_lcia_abbrev=['TTHH', 'TTEQ', 'm_CCS'],
            impact_abbrev=impact_abbrev,
            path=path_model_lca+'esm_results/',
            file_name=dat_file_name,
        )

### Second iteration: re-run the ESM with the first iteration results fed back in the LCI database

In [ ]:
for obj in ['TotalLCIA_m_CCS', 'TotalCost', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']:

    for background in ['base', 'esm_not_harmonized', 'esm_harmonized']:

        if background == 'base':
            impact_scores = pd.read_csv(f'./results/impact_scores.csv')
        elif background == 'esm_not_harmonized':
            impact_scores = pd.read_csv(f'./results/2050/impact_scores_{obj.replace("Total", "").replace("LCIA_", "").lower()}_wo_h.csv')
        elif background == 'esm_harmonized':
            impact_scores = pd.read_csv(f'./results/2050/impact_scores_{obj.replace("Total", "").replace("LCIA_", "").lower()}.csv')
        else:
            raise ValueError("Background must be one of 'base', 'esm_not_harmonized', or 'esm_harmonized'.")

        impact_scores.Impact_category = impact_scores.Impact_category.apply(lambda x: ast.literal_eval(x))

        es = run_esm(obj+'[2050]', scenario=True, lca_metrics_background=background, returns='model')
        results = es.calc()

        df_f_mult, df_annual_prod, df_annual_res = get_impact_scores(
            df_results=results,
            df_impact_scores=impact_scores,
            impact_category=list(impact_scores['Impact_category'].unique()),
            specific_year=2050,
        )

        df_f_mult['Run'] = f'{obj}_{background}'
        df_annual_prod['Run'] = f'{obj}_{background}'
        df_annual_res['Run'] = f'{obj}_{background}'

        df_f_mult.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)
        df_annual_prod.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)
        df_annual_res.rename(columns={'index0': 'Name', 'index1': 'Year'}, inplace=True)

        list_esm_results_f_mult.append(df_f_mult)
        list_esm_results_annual_prod.append(df_annual_prod)
        list_esm_results_annual_res.append(df_annual_res)

esm_results_f_mult = pd.concat(list_esm_results_f_mult)
esm_results_annual_prod = pd.concat(list_esm_results_annual_prod)
esm_results_annual_res = pd.concat(list_esm_results_annual_res)

To compare those results to what would typically be obtained with an ESM not coupled to LCA, we compute the impact obtained from direct CO2 emissions only.

In [ ]:
esm_db = Database('Tatooine_2050')

In [ ]:
df_flows_set_to_zero = pd.read_csv(path_results+'2050/removed_flows_list.csv')

In [ ]:
# Retrieving direct CO2 emissions
direct_co2_emissions_dict = {}

for tech in mapping.Name.unique():
    direct_co2_emissions_dict[tech] = 0

for tech in df_flows_set_to_zero.Name.unique():
    act_name_list =  df_flows_set_to_zero[df_flows_set_to_zero.Name == tech]['Activity'].unique()
    for act_name in act_name_list:
        act = [i for i in esm_db.db_as_list if i['name'] == act_name][0]
        direct_co2_emissions = sum([exc['amount'] for exc in act['exchanges'] if exc['name'] == 'Carbon dioxide, fossil'])
        direct_co2_emissions_dict[tech] += direct_co2_emissions

In [ ]:
esm_results_annual_prod['Direct CO2 emissions'] = esm_results_annual_prod['Annual_Prod'] * esm_results_annual_prod.Name.apply(lambda x: direct_co2_emissions_dict[x])

for imp_cat in ['Climate change, short term', 'Total human health', 'Total ecosystem quality']:
    # Adding direct CO2 emissions columns to the operation impacts dataframe
    esm_results_annual_prod[f'{imp_cat} (direct)'] = esm_results_annual_prod['Direct CO2 emissions'] * cf_dict[imp_cat]

    # The construction and resources impact are not modelled in ESMs without LCA
    esm_results_f_mult[f'{imp_cat} (direct)'] = 0
    esm_results_annual_res[f'{imp_cat} (direct)'] = 0

In [ ]:
esm_results_f_mult.to_csv('./results/soo_results_f_mult.csv', index=False)
esm_results_annual_prod.to_csv('./results/soo_results_annual_prod.csv', index=False)
esm_results_annual_res.to_csv('./results/soo_results_annual_res.csv', index=False)

## Validation direct CO2 emissions and primary energy use

In [ ]:
# To skip previous steps
R_long_direct = pd.read_csv(f'{path_results}impact_scores_direct.csv')
esm_results_annual_prod = pd.read_csv('./results/soo_results_annual_prod.csv')
esm.results_path_file = path_results + '2050/'

In [ ]:
esm.validation_direct_carbon_emissions(
    R_direct=R_long_direct,
    lcia_method_carbon_emissions=('IMPACT World+ Midpoint 2.1 for ecoinvent v3.10', 'Midpoint', 'Climate change, short term'),
    carbon_flow_in_esm='CO2',
    esm_results=esm_results_annual_prod[esm_results_annual_prod.Annual_Prod != 0].rename(columns={'Annual_Prod': 'Production'}),
)

In [ ]:
df_tot = esm.validation_double_counting(
    esm_results=esm_results_annual_prod[esm_results_annual_prod.Annual_Prod != 0].rename(columns={'Annual_Prod': 'Production'}),
)

In [ ]:
df_tot.to_csv(f'{path_results}2050/validation_primary_energy_system.csv', index=False)

## MOO

In [ ]:
def run_moo(obj1: str, obj2: str, main_variables_results: pd.DataFrame, N_run: int = 100):

    obj1_min = main_variables_results[main_variables_results['Objective'] == obj1][obj1].values[0]
    obj1_max = main_variables_results[main_variables_results['Objective'] == obj2][obj1].values[0]
    normalized_limit_list = list(np.linspace(obj1_min, obj1_max, N_run))

    data = ['limit_lcia', obj1.split('TotalLCIA_')[-1], 2050, None, None] + normalized_limit_list
    columns = ['param', 'index0', 'index1', 'index2', 'index3'] + [f'value{i+1}' for i in range(N_run)]

    seq_data = pd.DataFrame(data).T
    seq_data.columns = columns

    es = run_esm(objective_function=obj2+'[2050]', returns='model')

    results_pareto = es.calc_sequence(seq_data)

    return results_pareto

### TTEQ vs total cost

In [ ]:
N_run = 100
obj1 = 'TotalLCIA_TTEQ'
obj2 = 'TotalCost'
results_pareto_tteq_tc = run_moo(obj1, obj2, main_variables_results, N_run)

In [ ]:
# with open('./results/results_pareto_tteq_tc.pickle', 'wb') as f:
#     pickle.dump(results_pareto_tteq_tc, f)

In [ ]:
# with open('./results/results_pareto_tteq_tc.pickle', 'rb') as handle:
#     results_pareto_tteq_tc = pickle.load(handle)

In [ ]:
f_mult_pareto_tteq_tc = results_pareto_tteq_tc.variables['F_Mult'].reset_index()
f_mult_pareto_tteq_tc = f_mult_pareto_tteq_tc.merge(results_pareto_tteq_tc.variables['Annual_Prod'].reset_index(), how='left', on=['Run', 'index0', 'index1'])
lim_pareto_tteq_tc = results_pareto_tteq_tc.parameters['limit_lcia'].reset_index()
lim_pareto_tteq_tc = lim_pareto_tteq_tc[lim_pareto_tteq_tc.index0 == 'TTEQ'].drop(columns='index0')
f_mult_pareto_tteq_tc = f_mult_pareto_tteq_tc.merge(
    lim_pareto_tteq_tc,
    on=['Run', 'index1'],
    how='left',
)
f_mult_pareto_tteq_tc['limit_lcia'] *= 1e6 * max_tteq / N_cap
f_mult_pareto_tteq_tc.drop(columns=['index1'], inplace=True)
f_mult_pareto_tteq_tc = f_mult_pareto_tteq_tc[f_mult_pareto_tteq_tc.F_Mult != 0]
f_mult_pareto_tteq_tc = f_mult_pareto_tteq_tc[f_mult_pareto_tteq_tc.index0 != 'BATTERY']
f_mult_pareto_tteq_tc.index0 = f_mult_pareto_tteq_tc.index0.apply(lambda x: tech_name_dict[x] if x in tech_name_dict.keys() else x)
f_mult_pareto_tteq_tc = f_mult_pareto_tteq_tc[['Run', 'limit_lcia', 'index0', 'F_Mult', 'Annual_Prod']]
f_mult_pareto_tteq_tc.rename(columns={'index0': 'Technology', 'F_Mult': 'Installed capacity [GW]', 'Annual_Prod': 'Annual production [GWh]', 'limit_lcia': 'Upper limit on ecosystem quality damage [PDF.m2.yr/cap]'}, inplace=True)

In [ ]:
f_mult_pareto_tteq_tc.head()

### TTHH vs total cost

In [ ]:
N_run = 100
obj1 = 'TotalLCIA_TTHH'
obj2 = 'TotalCost'
results_pareto_tthh_tc = run_moo(obj1, obj2, main_variables_results, N_run)

In [ ]:
# with open('./results/results_pareto_tthh_tc.pickle', 'wb') as f:
#     pickle.dump(results_pareto_tthh_tc, f)

In [ ]:
# with open('./results/results_pareto_tthh_tc.pickle', 'rb') as handle:
#     results_pareto_tthh_tc = pickle.load(handle)

In [ ]:
f_mult_pareto_tthh_tc = results_pareto_tthh_tc.variables['F_Mult'].reset_index()
f_mult_pareto_tthh_tc = f_mult_pareto_tthh_tc.merge(results_pareto_tthh_tc.variables['Annual_Prod'].reset_index(), how='left', on=['Run', 'index0', 'index1'])
lim_pareto_tthh_tc = results_pareto_tthh_tc.parameters['limit_lcia'].reset_index()
lim_pareto_tthh_tc = lim_pareto_tthh_tc[lim_pareto_tthh_tc.index0 == 'TTHH'].drop(columns='index0')
f_mult_pareto_tthh_tc = f_mult_pareto_tthh_tc.merge(
    lim_pareto_tthh_tc,
    on=['Run', 'index1'],
    how='left',
)
f_mult_pareto_tthh_tc['limit_lcia'] *= 1e6 * max_tthh / N_cap
f_mult_pareto_tthh_tc.drop(columns=['index1'], inplace=True)
f_mult_pareto_tthh_tc = f_mult_pareto_tthh_tc[f_mult_pareto_tthh_tc.F_Mult != 0]
f_mult_pareto_tthh_tc = f_mult_pareto_tthh_tc[f_mult_pareto_tthh_tc.index0 != 'BATTERY']
f_mult_pareto_tthh_tc.index0 = f_mult_pareto_tthh_tc.index0.apply(lambda x: tech_name_dict[x] if x in tech_name_dict.keys() else x)
f_mult_pareto_tthh_tc = f_mult_pareto_tthh_tc[['Run', 'limit_lcia', 'index0', 'F_Mult', 'Annual_Prod']]
f_mult_pareto_tthh_tc.rename(columns={'index0': 'Technology', 'F_Mult': 'Installed capacity [GW]', 'Annual_Prod': 'Annual production [GWh]', 'limit_lcia': 'Upper limit on human health damage [DALY/cap]'}, inplace=True)

In [ ]:
f_mult_pareto_tthh_tc.head()

## Validation table

In [ ]:
val_double_counting = pd.read_csv(f'{esm.results_path_file}2050/validation_double_counting.csv')
val_efficiency = pd.read_csv(f'{esm.results_path_file}2050/efficiency_differences.csv')
val_cap_factor_cost = pd.read_csv(f'{esm.results_path_file}2050/capacity_factor_differences_cost.csv')
val_cap_factor_m_ccs = pd.read_csv(f'{esm.results_path_file}2050/capacity_factor_differences_m_ccs.csv')
val_cap_factor_tteq = pd.read_csv(f'{esm.results_path_file}2050/capacity_factor_differences_tteq.csv')
val_cap_factor_tthh = pd.read_csv(f'{esm.results_path_file}2050/capacity_factor_differences_tthh.csv')
val_direct_carbon_emissions = pd.read_csv(f'{esm.results_path_file}direct_carbon_emissions_differences.csv')

In [ ]:
val_double_counting = val_double_counting[['Name', 'Flow', 'ESM input quantity (ESM unit)', 'LCA input quantity (ESM unit)']].rename(columns={
    'Name': 'Technology',
    'ESM input quantity (ESM unit)': 'ESM',
    'LCA input quantity (ESM unit)': 'LCA',
})
val_double_counting['Step'] = 'Double-counting'
val_double_counting['Metric (unit)'] = 'Input quantity (kWh$_{fuel}$ / kWh$_{el}$)'

In [ ]:
val_efficiency = val_efficiency[['Name', 'Flow', 'ESM efficiency', 'LCA efficiency']].rename(columns={
    'Name': 'Technology',
    'ESM efficiency': 'ESM',
    'LCA efficiency': 'LCA',
})
val_efficiency['Step'] = 'Efficiency harmonization'
val_efficiency['ESM'] *= 100
val_efficiency['LCA'] *= 100
val_efficiency['Metric (unit)'] = 'Efficiency (\%)'

In [ ]:
val_cap_factor_cost = val_cap_factor_cost[['Name', 'Amount ESM', 'Amount LCA']].rename(columns={
    'Name': 'Technology',
    'Amount ESM': 'ESM',
    'Amount LCA': 'LCA',
})
val_cap_factor_cost['Objective function'] = 'TC'

val_cap_factor_m_ccs = val_cap_factor_m_ccs[['Name', 'Amount ESM', 'Amount LCA']].rename(columns={
    'Name': 'Technology',
    'Amount ESM': 'ESM',
    'Amount LCA': 'LCA',
})
val_cap_factor_m_ccs['Objective function'] = 'CCST'

val_cap_factor_tteq = val_cap_factor_tteq[['Name', 'Amount ESM', 'Amount LCA']].rename(columns={
    'Name': 'Technology',
    'Amount ESM': 'ESM',
    'Amount LCA': 'LCA',
})
val_cap_factor_tteq['Objective function'] = 'TTEQ'

val_cap_factor_tthh = val_cap_factor_tthh[['Name', 'Amount ESM', 'Amount LCA']].rename(columns={
    'Name': 'Technology',
    'Amount ESM': 'ESM',
    'Amount LCA': 'LCA',
})

val_cap_factor_tthh['Objective function'] = 'TTHH'

In [ ]:
val_direct_carbon_emissions = val_direct_carbon_emissions[['Name', 'ESM direct carbon emissions (ESM unit)', 'LCA direct carbon emissions (ESM unit)']].rename(columns={
    'Name': 'Technology',
    'ESM direct carbon emissions (ESM unit)': 'ESM',
    'LCA direct carbon emissions (ESM unit)': 'LCA',
})
val_direct_carbon_emissions['Step'] = 'Direct carbon emissions'
val_direct_carbon_emissions['Metric (unit)'] = 'Direct carbon emissions (kg CO$_2$-eq / kWh$_{el}$)'

In [ ]:
validation_table = pd.concat([
    val_double_counting,
    val_efficiency,
    val_direct_carbon_emissions
], axis=0, ignore_index=True)

validation_table['Objective function'] = 'All'
validation_table.Flow = validation_table.Flow.str.replace("['", '')
validation_table.Flow = validation_table.Flow.str.replace("']", '')

val_cap_factor = pd.concat([
    val_cap_factor_cost,
    val_cap_factor_m_ccs,
    val_cap_factor_tteq,
    val_cap_factor_tthh,
], axis=0, ignore_index=True)
val_cap_factor['Step'] = 'Capacity factor harmonization'
val_cap_factor['Metric (unit)'] = 'Capacity factor (unit / kWh$_{el}$)'

validation_table = pd.concat([validation_table, val_cap_factor], axis=0, ignore_index=True)

validation_table.Flow = validation_table.Flow.fillna('N/A')
validation_table.Flow = validation_table.Flow.apply(lambda x: tech_name_dict[x])
validation_table.Technology = validation_table.Technology.apply(lambda x: tech_name_dict[x])

validation_table['Delta'] = validation_table['ESM'] - validation_table['LCA']
validation_table['Delta rel'] = round(100 * validation_table['Delta'] / validation_table['LCA'], 1)
validation_table[['ESM', 'LCA', 'Delta']] = validation_table[['ESM', 'LCA', 'Delta']].map(lambda x: f"{x:.2E}")

validation_table = validation_table[['Step', 'Objective function', 'Metric (unit)', 'Technology', 'Flow', 'ESM', 'LCA', 'Delta', 'Delta rel']]

In [ ]:
validation_table.to_csv(f'{esm.results_path_file}2050/validation_table.csv', index=False)

## Visualize the results

In [ ]:
save_fig = False

In [ ]:
tech_to_show_list = [i for i in impact_scores.Name.unique() if i not in ['GRID']]

In [ ]:
esm_results_f_mult = pd.read_csv('./results/soo_results_f_mult.csv')
esm_results_annual_prod = pd.read_csv('./results/soo_results_annual_prod.csv')
esm_results_annual_res = pd.read_csv('./results/soo_results_annual_res.csv')

In [ ]:
run_first_it = ['TotalCost', 'TotalLCIA_m_CCS', 'TotalLCIA_TTHH', 'TotalLCIA_TTEQ']
run_second_it = [i for i in esm_results_f_mult.Run.unique() if i not in run_first_it]

### SOO

In [ ]:
mask_f_mult = (esm_results_f_mult.Run.isin(run_first_it)) & (esm_results_f_mult.Year == 2050)
mask_annual_prod = (esm_results_annual_prod.Run.isin(run_first_it)) & (esm_results_annual_prod.Year == 2050)
mask_annual_res = (esm_results_annual_res.Run.isin(run_first_it)) & (esm_results_annual_res.Year == 2050)

#### Installed capacities and annual productions

In [ ]:
df_cap = plot_energy_system_configuration(
    type='capacity',
    df_res=esm_results_f_mult[mask_f_mult],
    save_fig=save_fig,
)

In [ ]:
df_prod = plot_energy_system_configuration(
    type='production',
    df_res=esm_results_annual_prod[mask_annual_prod],
    save_fig=save_fig,
)

In [ ]:
df_syst_config = pd.merge(
    df_cap,
    df_prod,
    on=['Name', 'Run'],
    how='outer',
).rename(columns={'F_Mult': 'Installed capacity [GW]', 'Annual_Prod': 'Annual production [GWh]', 'Run': 'Objective function', 'Name': 'Technology'})
df_syst_config['Objective function'] = df_syst_config['Objective function'].apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
df_syst_config = df_syst_config.sort_values(by=['Objective function', 'Technology'])[['Objective function', 'Technology', 'Installed capacity [GW]', 'Annual production [GWh]']]

#### Contribution per technology

In [ ]:
plot_technologies_contribution(
    cat='Total human health',
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

In [ ]:
plot_technologies_contribution(
    cat='Total ecosystem quality',
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

In [ ]:
plot_technologies_contribution(
    cat='Climate change, short term',
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

In [ ]:
plot_technologies_contribution(
    cat='Total cost',
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    tech_to_show_list=tech_to_show_list,
    save_fig=save_fig,
)

Table of results

In [ ]:
list_esm_results_total = []

for obj in ['Total cost', 'Total human health', 'Total ecosystem quality', 'Climate change, short term']:
    esm_results_total = aggregate_phases_results(
        cat=obj,
        esm_results_f_mult=esm_results_f_mult[mask_f_mult],
        esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
        esm_results_annual_res=esm_results_annual_res[mask_annual_res],
        tech_to_show_list=tech_to_show_list,
    )
    esm_results_total['Metric'] = obj
    esm_results_total.rename(columns={obj: 'Value (total)', f'{obj} (direct)': 'Value (direct CO2)', 'Name': 'Technology or resource'}, inplace=True)
    list_esm_results_total.append(esm_results_total)
esm_results_total = pd.concat(list_esm_results_total)
esm_results_total = esm_results_total[esm_results_total['Value (total)'] != 0]
esm_results_total.Run = esm_results_total.Run.apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
esm_results_total.rename(columns={'Run': 'Objective function'}, inplace=True)
esm_results_total['Unit'] = esm_results_total.Metric.apply(lambda x: unit_ind_txt_dict[x]+'/(cap.yr)')
esm_results_total = esm_results_total.sort_values(['Metric', 'Objective function', 'Technology or resource'])[['Metric', 'Unit', 'Objective function', 'Technology or resource', 'Value (total)', 'Value (direct CO2)']]

#### Contribution per impact category

In [ ]:
impact_scores = pd.read_csv(path_results+'impact_scores.csv')
impact_scores.Impact_category = impact_scores.Impact_category.apply(lambda x: ast.literal_eval(x))

In [ ]:
hh_cat = [cat[2] for cat in impact_scores['Impact_category'].unique() if cat[1] == 'Human health' and cat[2] != 'Total human health']
eq_cat = [cat[2] for cat in impact_scores['Impact_category'].unique() if cat[1] == 'Ecosystem quality' and cat[2] != 'Total ecosystem quality']

In [ ]:
esm_results_f_mult_imp_cat_hh = esm_results_f_mult[mask_f_mult][['Name', 'Run'] + hh_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Construction').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')
esm_results_annual_prod_imp_cat_hh = esm_results_annual_prod[mask_annual_prod][['Name', 'Run'] + hh_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Operation').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')
esm_results_annual_res_imp_cat_hh = esm_results_annual_res[mask_annual_res][['Name', 'Run'] + hh_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Resource').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')

In [ ]:
esm_results_f_mult_imp_cat_eq = esm_results_f_mult[mask_f_mult][['Name', 'Run'] + eq_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Construction').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')
esm_results_annual_prod_imp_cat_eq = esm_results_annual_prod[mask_annual_prod][['Name', 'Run'] + eq_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Operation').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')
esm_results_annual_res_imp_cat_eq = esm_results_annual_res[mask_annual_res][['Name', 'Run'] + eq_cat].melt(id_vars=['Name', 'Run'], var_name='Impact category', value_name='Resource').groupby(['Run', 'Impact category']).sum().reset_index().drop(columns='Name')

In [ ]:
esm_results_imp_cat_hh = pd.merge(esm_results_f_mult_imp_cat_hh, esm_results_annual_prod_imp_cat_hh, on=['Run', 'Impact category'], how='outer')
esm_results_imp_cat_hh = pd.merge(esm_results_imp_cat_hh, esm_results_annual_res_imp_cat_hh, on=['Run', 'Impact category'], how='outer')

esm_results_imp_cat_eq = pd.merge(esm_results_f_mult_imp_cat_eq, esm_results_annual_prod_imp_cat_eq, on=['Run', 'Impact category'], how='outer')
esm_results_imp_cat_eq = pd.merge(esm_results_imp_cat_eq, esm_results_annual_res_imp_cat_eq, on=['Run', 'Impact category'], how='outer')

In [ ]:
esm_results_imp_cat_hh['Total'] = esm_results_imp_cat_hh['Construction'] + esm_results_imp_cat_hh['Operation'] + esm_results_imp_cat_hh['Resource']
esm_results_imp_cat_eq['Total'] = esm_results_imp_cat_eq['Construction'] + esm_results_imp_cat_eq['Operation'] + esm_results_imp_cat_eq['Resource']

In [ ]:
for col in ['Construction', 'Operation', 'Resource', 'Total']:
    esm_results_imp_cat_hh[col] = esm_results_imp_cat_hh[col] * 1e6 / N_cap
    esm_results_imp_cat_eq[col] = esm_results_imp_cat_eq[col] * 1e6 / N_cap

In [ ]:
esm_results_imp_cat_hh['Run'] = esm_results_imp_cat_hh['Run'].apply(lambda x: obj_code_dict[x])
esm_results_imp_cat_eq['Run'] = esm_results_imp_cat_eq['Run'].apply(lambda x: obj_code_dict[x])

In [ ]:
fig = px.bar(
    esm_results_imp_cat_hh,
    x='Run',
    y='Total',
    color='Impact category',
    barmode='stack',
    labels={
        'Impact category': 'Impact category',
        'Total': 'Human health damage [DALY/(cap.yr)]',
        'Run': 'Objective function',
    },
    width=650,
    height=525,
)

fig.for_each_trace(lambda t: t.update(marker_color=impact_category_hh_colors.get(t.name, '#000000')))
fig.update_layout(legend_traceorder='reversed')

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),  # left, right, top, bottom
)

if save_fig:
    fig.write_image('./figures/soo_impact_contrib_human_health.pdf')

fig.show()

In [ ]:
fig = px.bar(
    esm_results_imp_cat_eq,
    x='Run',
    y='Total',
    color='Impact category',
    barmode='stack',
    labels={
        'Impact category': 'Impact category',
        'Total': 'Ecosystem quality damage [PDF.m<sup>2</sup>.yr/(cap.yr)]',
        'Run': 'Objective function',
    },
    width=680,
    height=525,
)

fig.for_each_trace(lambda t: t.update(marker_color=impact_category_eq_colors.get(t.name, '#000000')))
fig.update_layout(legend_traceorder='reversed')

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),  # left, right, top, bottom
)

if save_fig:
    fig.write_image('./figures/soo_impact_contrib_ecosystem_quality.pdf')

fig.show()

Table of results

In [ ]:
esm_results_imp_cat_eq['Area of protection'] = 'Ecosystem quality'
esm_results_imp_cat_eq['Unit'] = 'PDF.m2.yr/(cap.yr)'
esm_results_imp_cat_hh['Area of protection'] = 'Human health'
esm_results_imp_cat_hh['Unit'] = 'DALY/(cap.yr)'
esm_results_imp_cat = pd.concat([
    esm_results_imp_cat_eq[['Area of protection', 'Unit', 'Run', 'Impact category', 'Total']],
    esm_results_imp_cat_hh[['Area of protection', 'Unit', 'Run', 'Impact category', 'Total']]
])
esm_results_imp_cat.Run = esm_results_imp_cat.Run.apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
esm_results_imp_cat.rename(columns={'Total': 'Value', 'Run': 'Objective function'}, inplace=True)

#### Contribution per elementary flows

In [ ]:
contrib_analysis_ef = pd.read_csv(path_results+'contribution_analysis.csv')

In [ ]:
def get_emissions_info(row):
    flow = bd.Database(row['database']).get(row['code'])
    return flow.as_dict()['name'], flow.as_dict()['categories']

In [ ]:
contrib_analysis_ef[['ef_name', 'ef_categories']] = pd.DataFrame(
    contrib_analysis_ef.apply(lambda x: get_emissions_info(x), axis=1).tolist(),
    index=contrib_analysis_ef.index
)

In [ ]:
df_ef_contrib_eq = plot_ef_contributions(
    df_contrib_analysis_ef=contrib_analysis_ef,
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    main_variables_results=main_variables_results,
    aop='Total ecosystem quality',
    save_fig=save_fig,
)

In [ ]:
df_ef_contrib_hh =plot_ef_contributions(
    df_contrib_analysis_ef=contrib_analysis_ef,
    esm_results_f_mult=esm_results_f_mult[mask_f_mult],
    esm_results_annual_prod=esm_results_annual_prod[mask_annual_prod],
    esm_results_annual_res=esm_results_annual_res[mask_annual_res],
    main_variables_results=main_variables_results,
    aop='Total human health',
    save_fig=save_fig,
)

Table of results

In [ ]:
df_ef_contrib_eq['Area of protection'] = 'Ecosystem quality'
df_ef_contrib_eq['Unit'] = 'PDF.m2.yr/(cap.yr)'
df_ef_contrib_hh['Area of protection'] = 'Human health'
df_ef_contrib_hh['Unit'] = 'DALY/(cap.yr)'
df_ef_contrib = pd.concat([
    df_ef_contrib_eq[['Area of protection', 'Unit', 'Run', 'ef_name', 'scaled_impact']],
    df_ef_contrib_hh[['Area of protection', 'Unit', 'Run', 'ef_name', 'scaled_impact']]
])
df_ef_contrib.Run = df_ef_contrib.Run.apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
df_ef_contrib.rename(columns={'scaled_impact': 'Value', 'Run': 'Objective function', 'ef_name': 'Elementary flow'}, inplace=True)
df_ef_contrib.sort_values(['Area of protection', 'Objective function', 'Elementary flow'], inplace=True)

### SOO with ESM results

In [ ]:
esm_results_f_mult = pd.read_csv('./results/soo_results_f_mult.csv')
esm_results_annual_prod = pd.read_csv('./results/soo_results_annual_prod.csv')
esm_results_annual_res = pd.read_csv('./results/soo_results_annual_res.csv')

In [ ]:
df_cap_sec = plot_technologies_contribution_second_iteration(
    type='capacity',
    df_res=esm_results_f_mult,
    save_fig=save_fig,
)

In [ ]:
df_prod_sec = plot_technologies_contribution_second_iteration(
    type='production',
    df_res=esm_results_annual_prod,
    save_fig=save_fig,
)

In [ ]:
df_syst_config_sec = pd.merge(df_cap_sec[['Run', 'Background database', 'Name', 'F_Mult']], df_prod_sec[['Run', 'Background database', 'Name', 'Annual_Prod']])
df_syst_config_sec = df_syst_config_sec[df_syst_config_sec.F_Mult != 0]
df_syst_config_sec.rename(columns={'F_Mult': 'Installed capacity [GW]', 'Annual_Prod': 'Annual production [GWh]', 'Run': 'Objective function', 'Name': 'Technology'}, inplace=True)
df_syst_config_sec['Objective function'] = df_syst_config_sec['Objective function'].apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)

In [ ]:
list_of_dfs = [esm_results_f_mult, esm_results_annual_prod, esm_results_annual_res]

for i, df in enumerate(list_of_dfs):
    filtered_df = df[df['Run'].isin(run_second_it)].copy()
    filtered_df['Background database'] = filtered_df['Run'].apply(harmonization_level)
    filtered_df['Run'] = filtered_df['Run'].str.replace('_base', '', regex=False)
    filtered_df['Run'] = filtered_df['Run'].str.replace('_esm_harmonized', '', regex=False)
    filtered_df['Run'] = filtered_df['Run'].str.replace('_esm_not_harmonized', '', regex=False)
    list_of_dfs[i] = filtered_df

esm_results_f_mult, esm_results_annual_prod, esm_results_annual_res = list_of_dfs

In [ ]:
esm_results_total_tthh = aggregate_phases_results(
    cat='Total human health',
    esm_results_f_mult=esm_results_f_mult[(esm_results_f_mult.Year == 2050)],
    esm_results_annual_prod=esm_results_annual_prod[(esm_results_annual_prod.Year == 2050)],
    esm_results_annual_res=esm_results_annual_res[(esm_results_annual_res.Year == 2050)],
    tech_to_show_list=tech_to_show_list,
)

plot_technologies_contribution_second_iteration(
    type='impact',
    cat='Total human health',
    df_res=esm_results_total_tthh,
    save_fig=save_fig,
)

In [ ]:
esm_results_total_tteq = aggregate_phases_results(
    cat='Total ecosystem quality',
    esm_results_f_mult=esm_results_f_mult,
    esm_results_annual_prod=esm_results_annual_prod,
    esm_results_annual_res=esm_results_annual_res,
    tech_to_show_list=tech_to_show_list,
)

plot_technologies_contribution_second_iteration(
    type='impact',
    cat='Total ecosystem quality',
    df_res=esm_results_total_tteq,
    save_fig=save_fig,
)

In [ ]:
esm_results_total_ccst = aggregate_phases_results(
    cat='Climate change, short term',
    esm_results_f_mult=esm_results_f_mult,
    esm_results_annual_prod=esm_results_annual_prod,
    esm_results_annual_res=esm_results_annual_res,
    tech_to_show_list=tech_to_show_list,
)

plot_technologies_contribution_second_iteration(
    type='impact',
    cat='Climate change, short term',
    df_res=esm_results_total_ccst,
    save_fig=save_fig,
)

In [ ]:
esm_results_total_tthh['Metric'] = 'Total human health'
esm_results_total_tteq['Metric'] = 'Total ecosystem quality'
esm_results_total_ccst['Metric'] = 'Climate change, short term'
esm_results_total_tthh['Unit'] = 'DALY/(cap.yr)'
esm_results_total_tteq['Unit'] = 'PDF.m2.yr/(cap.yr)'
esm_results_total_ccst['Unit'] = 't CO2-eq/(cap.yr)'

esm_results_total_tthh.rename(columns={'Total human health': 'Value (total)', 'Total human health (direct)': 'Value (direct CO2)'}, inplace=True)
esm_results_total_tteq.rename(columns={'Total ecosystem quality': 'Value (total)', 'Total ecosystem quality (direct)': 'Value (direct CO2)'}, inplace=True)
esm_results_total_ccst.rename(columns={'Climate change, short term': 'Value (total)', 'Climate change, short term (direct)': 'Value (direct CO2)'}, inplace=True)

esm_results_total_sec = pd.concat([
    esm_results_total_tthh[['Metric', 'Unit', 'Run', 'Background database', 'Name', 'Value (total)', 'Value (direct CO2)']],
    esm_results_total_tteq[['Metric', 'Unit', 'Run', 'Background database', 'Name', 'Value (total)', 'Value (direct CO2)']],
    esm_results_total_ccst[['Metric', 'Unit', 'Run', 'Background database', 'Name', 'Value (total)', 'Value (direct CO2)']],
])

esm_results_total_sec = esm_results_total_sec[esm_results_total_sec['Value (total)'] != 0]
esm_results_total_sec.Run = esm_results_total_sec.Run.apply(lambda x: full_name_ind[x] if x in full_name_ind.keys() else x)
esm_results_total_sec.rename(columns={'Run': 'Objective function', 'Name': 'Technology or resource'}, inplace=True)

esm_results_total_sec = esm_results_total_sec.sort_values(['Metric', 'Objective function', 'Technology or resource'])[['Metric', 'Unit', 'Objective function', 'Background database', 'Technology or resource', 'Value (total)', 'Value (direct CO2)']]

### MOO

#### Pareto front

In [ ]:
plot_pareto_front(results_pareto_tteq_tc, main_variables_results, 'TotalLCIA_TTEQ', 'TotalCost', 'TotalLCIA_TTHH')

In [ ]:
plot_pareto_front(results_pareto_tthh_tc, main_variables_results, 'TotalLCIA_TTHH', 'TotalCost', 'TotalLCIA_TTEQ')

#### MOO indicators

In [ ]:
df_ind_pareto_tteq = plot_moo_indicators(results_pareto_tteq_tc, 'TotalLCIA_TTEQ', direct_co2_emissions_dict, save_fig=save_fig)

In [ ]:
df_ind_pareto_tthh = plot_moo_indicators(results_pareto_tthh_tc, 'TotalLCIA_TTHH', direct_co2_emissions_dict, save_fig=save_fig)

#### MOO configurations

In [ ]:
plot_moo_config(results_pareto_tteq_tc, 'TotalLCIA_TTEQ', y_axis='Annual_Prod', plot_type='stack_plot', save_fig=save_fig)

In [ ]:
plot_moo_config(results_pareto_tteq_tc, 'TotalLCIA_TTEQ', y_axis='F_Mult', plot_type='stack_plot', save_fig=save_fig)

In [ ]:
plot_moo_config(results_pareto_tthh_tc, 'TotalLCIA_TTHH', y_axis='Annual_Prod', plot_type='stack_plot', save_fig=save_fig)

In [ ]:
plot_moo_config(results_pareto_tthh_tc, 'TotalLCIA_TTHH', y_axis='F_Mult', plot_type='stack_plot', save_fig=save_fig)

## Save all results dataframes to an Excel file

In [ ]:
with pd.ExcelWriter('results/SI_results_figures.xlsx', engine='xlsxwriter') as writer:
    workbook = writer.book

    # First iteration - energy system configurations for SOOs
    name = 'Table S1'
    writer.sheets[name] = workbook.add_worksheet(name)
    df_syst_config.to_excel(writer, sheet_name=name, index=False, startrow=0, startcol=0)

    # First iteration - contributions of technologies for SOOs
    name = 'Table S2'
    writer.sheets[name] = workbook.add_worksheet(name)
    esm_results_total.to_excel(writer, sheet_name=name, index=False)

    # First iteration - energy system configurations for HH MOO
    name = 'Table S3'
    writer.sheets[name] = workbook.add_worksheet(name)
    f_mult_pareto_tthh_tc.to_excel(writer, sheet_name=name, index=False)

    # First iteration - energy system configurations for EQ MOO
    name = 'Table S4'
    writer.sheets[name] = workbook.add_worksheet(name)
    f_mult_pareto_tteq_tc.to_excel(writer, sheet_name=name, index=False)

    # First iteration - contributions of technologies for HH MOO
    name = 'Table S5'
    writer.sheets[name] = workbook.add_worksheet(name)
    df_ind_pareto_tthh.to_excel(writer, sheet_name=name, index=False)

    # First iteration - contributions of technologies for EQ MOO
    name = 'Table S6'
    writer.sheets[name] = workbook.add_worksheet(name)
    df_ind_pareto_tteq.to_excel(writer, sheet_name=name, index=False)

    # First iteration - contributions of impact categories
    name = 'Table S7'
    writer.sheets[name] = workbook.add_worksheet(name)
    esm_results_imp_cat.to_excel(writer, sheet_name=name, index=False)

    # First iteration - contributions of elementary flows
    name = 'Table S8'
    writer.sheets[name] = workbook.add_worksheet(name)
    df_ef_contrib.to_excel(writer, sheet_name=name, index=False)

    # Second iteration - energy system configurations
    name = 'Table S9'
    writer.sheets[name] = workbook.add_worksheet(name)
    df_syst_config_sec.to_excel(writer, sheet_name=name, index=False)

    # Second iteration - contributions of technologies
    name = 'Table S10'
    writer.sheets[name] = workbook.add_worksheet(name)
    esm_results_total_sec.to_excel(writer, sheet_name=name, index=False)